A) Utility: compute discounted returns (Monte Carlo-style)

In [ ]:
import numpy as np

def discounted_return(rewards, gamma=0.99):
    """
    rewards: list/array of rewards r_t, r_{t+1}, ...
    returns G_0 for the whole trajectory.
    """
    G = 0.0
    power = 1.0
    for r in rewards:
        G += power * r
        power *= gamma
    return G